In [42]:
import numpy as np
from scipy import signal
import matplotlib
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

# define Activation layers and derivatives
Sigmoid = lambda x: 1.0/(1.0000000000001 + np.exp(-x))
Sigmoid_delta = lambda x, Sigmoid: Sigmoid(x)*(1-Sigmoid(x))
Relu = lambda x: np.maximum(x, 0.0)
Relu_delta = lambda x: np.maximum(x+0.0001, 0.0)/np.maximum(x, 0.0001)-0.0001
# define loss layers
Softmax = lambda x: np.exp(x)/np.nansum(np.exp(x), axis=0)
one_hot = lambda labels, n_classes: np.eye(n_classes)[np.array(labels).reshape(-1)].transpose()
ce_loss = lambda labels,output: -np.nansum(np.log(Softmax(output))* one_hot(labels, output.shape[0]), axis=0)
ce_softmax_delta = lambda labels,output: output - one_hot(labels, output.shape[0])

Define a Max Pooling layer with downsampling integer size. Reshapes and takes the maximum across the downsampled reshape dimension.

In [19]:
MaxPool = lambda data, ds=2: data[:, :(data.shape[1] // ds)*ds, :(data.shape[2] // ds)*ds].reshape(data.shape[0], data.shape[1] // ds, ds, data.shape[2] // ds, ds).max(axis=(2, 4))

# Convolutional neural networks

Open a single example image dataset: the number 5 from the MNIST data set.

In [20]:
imgfile='data/number5.jpg'
from PIL import Image
img_pil = Image.open(imgfile)
img_pil_32x32= img_pil.resize((32,32), Image.LINEAR)
# reshape as having a single input channel
# we're going to be working with input and output channels as we go thru the network
data = np.asarray(img_pil_32x32).astype(np.float32).reshape((32,32,1))
labels=np.asarray([[5]])

conv weights are in form channels_in (I), channels_out (J), kernel_width, kernel_height and convolution will proceed with one kernel (e.g. $weight[i,j,:,:]$) over each channel i of the I channels of data/activation flowing into this layer and these will be summed over all i to generate the jth output feature map/image, resulting in J output channels.

In [21]:
in_channels=1
out_channels=8
# typically (but not necessarily) the filter is symmetric
filter_size=5
weights = np.random.randn(in_channels, out_channels, filter_size, filter_size)
# Bias on the other hand is just like the fully-connecteds, there will be one bias per output channel
biases = np.random.randn(out_channels, 1)

In [22]:
# so we will be doing something like this:
output=[]
for j in range(out_channels):
    temp=None
    for i in range(in_channels):
        if temp is None:
            temp = signal.convolve2d(weights[i,j,:,:],data[:,:,i])
        else:
            temp += signal.convolve2d(weights[i,j,:,:],data[:,:,i])
    output.append(temp/in_channels + biases[j])
output = np.asarray(output)
print(data.shape)
print(output.shape)

(32, 32, 1)
(8, 36, 36)


Notice the channels dimension naturally is easier to work with if it comes first. Remember we'll be dealing with larger networks and each feature map, activation map should have the same structure. So at this point, we should permute the data so channels is the first dimension.

In [23]:
data = data.transpose((2, 0, 1))
print(data.shape)

(1, 32, 32)


# Convolutional layer and its variants

In [24]:
# one-liner with list comprehension
Conv2D = lambda data, weights, biases: \
    np.asarray([sum( \
        [signal.convolve2d(data[i,:,:], weights[i,j,:,:].squeeze(), mode='same') \
         for i in range(weights.shape[0])]) \
        + biases[j] for j in range(weights.shape[1])])

## Padding
signal.convolve2d has three modes, corresponding to common practices when using convolutions. Strictly, the traditional convolution is 'full' convolution, but we usually use 'same' or 'valid' modes.

'same' mode means a constant padding (zeros, but can be adjusted) is applied to expand the image so the output will end up the same size as the input. This creates a padding = (filter_size//2)

'valid' mode means the filter is only applied in regions of the image where the kernel can be contained, meaning the edges will get cut off
         size_output = size_input - 2*(filter_size//2)

'full' is the default, meaning the mathematical convolution will be applied, which will expand the image to
         size_output = size_input + 2*(filter_size//2)


Note its not uncommon to skip biases for convolutional layers (usually with something like use_bias=False in one of the platforms)

In [26]:
# pass data thru conv layer defined by the weights and biases
output = Conv2D(data, weights, biases)
print('Filter-matched padding, input data shape '+str(data.shape)+', output data shape: '+str(output.shape))

Filter-matched padding, input data shape (1, 32, 32), output data shape: (8, 32, 32)


## Variation: padding

In [27]:
Conv2DNoPad = lambda data, weights, biases, pad=0: np.asarray([sum( [signal.convolve2d(data[i,:,:], weights[i,j,:,:].squeeze(), mode='valid') for i in range(weights.shape[0])] )+biases[j] for j in range(weights.shape[1])])
print('No padding, output data shape: '+str(Conv2DNoPad(data, weights, biases).shape))

No padding, output data shape: (8, 28, 28)


## Variation: stride
Please note, this is not a robust implementation, ideally we'd want to define starts/stops of strides, and limit needless computation.

In [28]:
Conv2DStride = lambda data, weights, biases, stride=2: np.asarray([sum( [signal.convolve2d(data[i,:,:], weights[i,j,:,:].squeeze(), mode='same') for i in range(weights.shape[0])] )+biases[j] for j in range(weights.shape[1])])[:,::stride,::stride]
print('Padding and stride==2, output data shape: '+str(Conv2DNoPad(data, weights, biases).shape))

Padding and stride==2, output data shape: (8, 28, 28)


## Variation: group
group convolution (with groups=channels) or depthwise convolution (with groups<channels), the number of filters must match the number of input channels and the output is same size (and channels) as input. Each filter is applied to ONLY one channel, meaning convolution operations are done separately per channel in this layer note, there are only C-channels, no input/output channels in the weights. ALSO note, from here on, we've redefined data to start with the channel dimension first

In [29]:
depthwise_weights = np.random.randn(out_channels, filter_size, filter_size)
Conv2DDepthwise = lambda data, weights, biases: np.asarray([signal.convolve2d(data[i,:,:], depthwise_weights[i,:,:].squeeze(), mode='same')+biases[i] for i in range(weights.shape[0])] )
print('Input to depthwise: '+str(output.shape)+', output: '+str(Conv2DDepthwise(output, depthwise_weights, biases).shape))

Input to depthwise: (8, 32, 32), output: (8, 32, 32)


## Variation: depth-separable convolution
Here, we do a MxN with C-groupwise (depthwise) convolution followed by a 1x1 regular convolution with as many channels as makes sense, here lets double the number.

In [30]:
weights_1x1 = np.random.randn(out_channels, 2*out_channels, 1, 1)
biases_1x1 = np.random.randn(2*out_channels, 1)
# however, scipy's convolve2d doesn't really support 1x1 (non-2D) convolutions, so we have to implement it differently
output_depthwise = Conv2DDepthwise(output, depthwise_weights, biases)
try:
    output_depth_sep = Conv2D(output_depthwise, weights_1x1, biases_1x1)
except:
    print('Nope, scipy convolve2d only supports actual 2D convolution')

Conv2D_1x1 = lambda data, weights, biases: np.asarray([sum([data[i,:,:]*weights[i, c, 0, 0] for i in range(weights.shape[0])]) for c in range(weights.shape[1])])
output_depth_sep = Conv2D_1x1(output_depthwise, weights_1x1, biases_1x1)
print('input to depth-separable pair of convolutions: '+str(output.shape)+', output: '+str(output_depth_sep.shape))

Nope, scipy convolve2d only supports actual 2D convolution
input to depth-separable pair of convolutions: (8, 32, 32), output: (16, 32, 32)


## Variation:dilated/atrous convolution
For this, we insert "holes" in the filter before applying, equivalent to expanding the filter and filling with zeros so a 3x3 filter with dilation=2 becomes a 5x5 without any added computation (assuming its implemented well, we're going to be lazy and keep using scipy's convolve2d), and a 3x3 with dilation=3 becomes a 7x7 filter. You could say it expands the "receptive area" of a filter.

In [31]:
dilation=2
dilated_filter_size = dilation*(filter_size-1)+1
dilated_weights = np.zeros( (weights.shape[0], weights.shape[1], (weights.shape[2]-1)*dilation + 1, (weights.shape[3]-1)*dilation + 1) )
dilated_weights[:, :, ::dilation, ::dilation]=weights
output_atrous = Conv2D(data, dilated_weights, biases)

## Variation: up-volution (fractional-strided convolution)
Apply the filter to the input data but here the input data is dilated (atrous) like above it is also called transposed convolution, and also sometimes called deconvolution, which it isn't, but this is what they're referring to.

In [32]:
fmap = output
dilated_fmap = np.zeros( (fmap.shape[0], fmap.shape[1]*dilation, fmap.shape[2]*dilation) )
dilated_fmap[:, ::dilation, ::dilation]=fmap
# note, like some of these layers, this implementation involves a choice of where to index from, so
# platforms can differ in the fine details without you knowing anything about it

# Convolutional network builder
For this network, we will combine convolutions and fully connected layers. We will need a "flatten" layer so the fully connected layers can accept the last convolutional layer, and we'll use maxpooling layers as well.

In [33]:
import collections
def get_CNN_params(w_scale=0.01, b_scale=0.0, activations=[], conv_sizes=[], maxpool_sizes=[], fc_sizes=[]):
    # default is lenet-5
    if len(conv_sizes)==0:
        conv_sizes=[(1,6,5,5), 0, (6,16,5,5), 0, 0, 0, 0]
        maxpool_sizes=[1, 2, 1, 2, 1, 1, 1]
        fc_sizes=[0, 0, 0, 0, 400, 120, 84, 10]
    if len(activations)==0:
        activations=[Sigmoid, None, Sigmoid, None, Sigmoid, Sigmoid, None]
    params=[]
    first_fc=True
    for i in range(len(fc_sizes)-1):
        layer=collections.namedtuple('layer',['type', 'pool','weight','bias','activation'])
        if fc_sizes[i]>0:
            if first_fc:
                first_fc=False
                layer=collections.namedtuple('layer',['type', 'pool','weight','bias','activation'])
                layer.type='Flatten'
                layer.activation = None
                params.append(layer)
                layer=collections.namedtuple('layer',['type', 'pool','weight','bias','activation'])
            layer.type='FullyConnected'
            layer.weights = w_scale * np.random.randn(fc_sizes[i+1], fc_sizes[i])
            print('layer weights ',layer.weights.shape)
            layer.bias  = b_scale * np.random.randn(fc_sizes[i+1], 1)
            print('layer bias ',layer.bias.shape)
        elif maxpool_sizes[i]>1:
            layer.type='MaxPool'
            layer.weight = None
            layer.bias  = None
        else:
            layer.type='Conv2D'
            layer.weights = w_scale * np.random.random(conv_sizes[i])
            layer.bias  = b_scale * np.random.randn(conv_sizes[i][1], 1)
        layer.activation = activations[i]
        params.append(layer)
    if params[-1].type=='Conv2D':
        # if last layer was Conv, append a flatten layer
        layer=collections.namedtuple('layer',['type', 'pool','weight','bias','activation'])
        layer.type='Flatten'
        layer.activation = None
        params.append(layer)
    return params

In [35]:
# setup network config, use valid pooling from here on in forward passes
Conv2D = lambda data, weights, biases: np.asarray([sum( [signal.convolve2d(data[i,:,:], weights[i,j,:,:].squeeze(), mode='valid') for i in range(weights.shape[0])] )+biases[j] for j in range(weights.shape[1])])
Conv2D_1x1 = lambda data, weights, biases: np.asarray([sum([data[i,:,:]*weights[i, c, 0, 0] for i in range(weights.shape[0])]) for c in range(weights.shape[1])])
activation_fn=Sigmoid
delta_activation_fn=Sigmoid_delta

In [37]:
# Example 1: fully convolutional (i.e., no fc layers)
# 32x32x1 -> 8x28x28 (ds to 14x14) -> 32x10x10 (ds to 5x5) -> 64x1x1 -> 10x1x1 -> flatten
params_fcn = get_CNN_params(activations=[activation_fn, None, activation_fn, \
                                         None, activation_fn, None], \
                            conv_sizes=[(1,8,5,5),0,(8,32,5,5),0,(32,64,5,5),(64,10,1,1)], \
                            maxpool_sizes=[1,2,1,2,1,1,1], fc_sizes=[0,0,0,0,0,0,0])

In [38]:
# Example 2: get the LeNet-5 network arch
params_lenet5 = get_CNN_params(activations=[activation_fn, None, activation_fn, None, activation_fn, activation_fn, None])

layer weights  (120, 400)
layer bias  (120, 1)
layer weights  (84, 120)
layer bias  (84, 1)
layer weights  (10, 84)
layer bias  (10, 1)


In [39]:
# create forward pass function capable of handling this layer-based param structure
def forward(data, params, forward_only=True):
    activations=[data]
    features=[]
    for layer in params:
        print(layer.type)
        print('activations flowing into this layer: ', activations[-1].shape)
        if layer.type == 'Flatten':
            features.append(activations[-1].reshape((-1, 1)))
            # next layer for dot product MUST have shape (N,1) not (N,)
            activations.append(features[-1])
            continue
        elif layer.type == 'Conv2D':
            if layer.weights.shape[-1]>1:
                features.append(Conv2D(activations[-1], layer.weights, layer.bias))
            else:
                features.append(Conv2D_1x1(activations[-1], layer.weights, layer.bias))
        elif layer.type == 'FullyConnected':
            features.append(np.dot(layer.weights, activations[-1]) + layer.bias)
        elif layer.type == 'MaxPool':
            features.append(MaxPool(activations[-1]))
        activations.append(layer.activation(features[-1]) if layer.activation is not None else features[-1])
    if forward_only:
        return activations[-1]
    else:
        return activations,features

# Backpropagation for convolutional network
Remembering that convolution is defined as a backwards operator of the filter over the local pixels. This means the 2D convolution is like flipping the weights in both axes and applying directly. Backpropagation for convolution is like with fully connected except the weights multiplication to get the current layer's gradient must be done with similarly flipped weights from the subsequent layer, and with flipped activation flowing from the current layer:

dL/dW = layer_gradient * activation(flipped activation)

In [41]:
# run on fully-convolutional network
activations,features = forward(data, params_fcn, forward_only=False)

softmax_outputs = Softmax(activations[-1])
loss = ce_loss(labels, activations[-1])
print('Single forward pass loss on initialized network: ',loss)

Conv2D
activations flowing into this layer:  (1, 32, 32)
MaxPool
activations flowing into this layer:  (8, 28, 28)
Conv2D
activations flowing into this layer:  (8, 14, 14)
MaxPool
activations flowing into this layer:  (32, 10, 10)
Conv2D
activations flowing into this layer:  (32, 5, 5)
Conv2D
activations flowing into this layer:  (64, 1, 1)
Flatten
activations flowing into this layer:  (10, 1, 1)
Single forward pass loss on initialized network:  [2.301]


In [44]:
# getting complex enough that we need to track the layers, weights and biases better
gradients={'layers': [], 'weights': [], 'biases': []}
# append the final layer gradient from what we know to be the log-likelihood+softmax gradient
gradients['layers'].append(softmax_outputs - one_hot(labels, softmax_outputs.shape[0]))